# Movie Recommender System using KNN
Hasti Hojabr \
ID : 97216040\
Proffesor: Marzie Farhang\
Course: Statics, Probability and Data Analysis
Date: May 30,2022

MovieLens - Movie Recommendation Data Sets http://www.grouplens.org/node/73

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
from IPython.display import display, HTML
warnings.filterwarnings("ignore")

In [2]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

In [3]:
def Recommender_Systems_PreProccessing(num_users,number_ratings,number_movies):
    
    path          = './ml-25m/{}.csv'
    movies        = pd.read_csv(path.format('movies'))
    ratings       = pd.read_csv(path.format('ratings'))
    
    def Reduction(num_users,number_ratings,number_movies):
        rate = ratings['rating'].unique()
        rate.sort()
        min_rating = len(rate)-number_ratings
        ratings_re = ratings[(ratings['rating']>min_rating)]            #reducing the dataset 
        #ID = ratings['userId'].value_counts(sort='True').index.tolist() #sorted by most active user to least active one
        #User = [str(i) for i in ID[:num_users]]
        #ratings_re = ratings_re.loc[ratings_re['userId'].isin(User)]
        print('Original lenght of ratings data set:,',len(ratings))
        print('Reduced lenght of ratings data set:,',len(ratings_re[:number_movies]))
        return ratings_re[:number_movies]
    
    def User_Movie_Matrix(movies,ratings_re):
        matrix_pivot = ratings_re.pivot_table(index='movieId', columns='userId', values='rating').fillna(0) 
        display(matrix_pivot)
        matrix = csr_matrix(matrix_pivot.values)
        print('Shape of Matrix:',matrix.shape)

        movie = movies.set_index('movieId').loc[matrix_pivot.index] 
        key,val = [],[]
        for i,j in enumerate(list(movie['title'])): key.append(j) ; val.append(i)
        dictt = dict(zip(key,val))
        return matrix, dictt

    def main():
        ratings_re = Reduction(num_users,number_ratings,number_movies)
        matrix, dictt = User_Movie_Matrix(movies,ratings_re)
        return matrix, dictt

    matrix, dictt = main()
    return matrix, dictt
    

In [4]:
%%time
###PreProccessing parameters
num_users      = 0
number_ratings = 8
number_movies  = 1000000

Matrix, Dictt = Recommender_Systems_PreProccessing(num_users,number_ratings,number_movies)

Original lenght of ratings data set:, 25000095
Reduced lenght of ratings data set:, 1000000


userId,1,2,3,4,5,6,7,8,9,10,...,7724,7725,7726,7727,7728,7729,7730,7731,7732,7733
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.5,4.0,3.0,4.0,0.0,0.0,4.0,0.0,3.5,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
208737,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
208793,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Shape of Matrix: (20865, 7732)
CPU times: user 20.9 s, sys: 4.97 s, total: 25.9 s
Wall time: 28.5 s


In [5]:
def Recommender_Systems(IMDb_link, Matrix, Dictt, Number_of_Recoms):
    
    path           = './ml-25m/{}.csv'
    movies         = pd.read_csv(path.format('movies'))
    links          = pd.read_csv(path.format('links'))
    
    def Find_movie_name(IMDb_Link):
        IMDb_Id    = int(''.join([n for n in IMDb_Link if n.isdigit()]))
        movie_ID   = links.loc[links.imdbId==IMDb_Id].movieId
        movie_name = movies.loc[movies.movieId==movie_ID.tolist()[0]].title
        return movie_name.tolist()[0]
    
    KNN = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=30, n_jobs=-1)
    KNN.fit(Matrix)

    Movie_name = Find_movie_name(IMDb_link) ; i = Dictt[Movie_name]
    distances, index = KNN.kneighbors(Matrix[i],Number_of_Recoms+1)

    dictt_rev = dict((v, k) for k, v in Dictt.items())
    
    name = [dictt_rev[i] for i in index[0]] ; dist = [d for d in distances[0]]
    name.reverse() ; dist.reverse()
    recom = pd.DataFrame(zip(name,dist),columns=['Recommended Movie name','Distance'])
    
    recom = recom.set_index('Recommended Movie name')
    recom.drop(Movie_name,inplace=True) 
    
    return print('Recommendations for ',"\033[1m" + Movie_name + "\033[0m",'.'),display(recom)

In [6]:
###Recommindation parameters
IMDb_link        = input('Please give IMDb link of your favorite movie.')
Number_of_Recoms = input('How many recommendations do you want to have?')

_,_ = Recommender_Systems(IMDb_link, Matrix, Dictt, int(Number_of_Recoms))

Please give IMDb link of your favorite movie.https://www.imdb.com/title/tt0097165/
How many recommendations do you want to have?5
Recommendations for  Dead Poets Society (1989) .


,Distance
Recommended Movie name,
Groundhog Day (1993),0.557362
Back to the Future (1985),0.549969
"Truman Show, The (1998)",0.533566
Rain Man (1988),0.517819
Good Will Hunting (1997),0.508767


In [7]:
###Recommindation parameters
IMDb_link        = input('Please give IMDb link of your favorite movie.')
Number_of_Recoms = input('How many recommendations do you want to have?')

_,_ = Recommender_Systems(IMDb_link, Matrix, Dictt, int(Number_of_Recoms))

Please give IMDb link of your favorite movie.https://www.imdb.com/title/tt0241527/
How many recommendations do you want to have?5
Recommendations for  Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001) .


,Distance
Recommended Movie name,
Harry Potter and the Half-Blood Prince (2009),0.441328
Harry Potter and the Order of the Phoenix (2007),0.408861
Harry Potter and the Goblet of Fire (2005),0.338485
Harry Potter and the Prisoner of Azkaban (2004),0.283338
Harry Potter and the Chamber of Secrets (2002),0.263741
